# BDA학회 데이터 분석 모델링반 (ML2) 3주차 복습과제

제출자 성명: 이승섭89

이전 수업과 같은 내용을 복습했습니다.

Python 3.10.14 버전을 사용합니다.

### SVM Analysis

$$\underset{\mathbf{w},b}\min \frac{1}{2}||\mathbf{w}||^2 \\ y_{i}(\mathbf{w} \cdot \mathbf{x}_{i}+b) \geq 1$$

- Hard margin SVM
    - When data is entirely linearly separable, margin is maximized without any errors.

$$\underset{\mathbf{w},b,\xi} \min \frac{1}{2} ||\mathbf{w}||^2+C\sum_{i=1}^{n}\xi_{i} \\ y_{i}(\mathbf{w} \cdot \mathbf{x}_{i} + b) \geq 1 - \xi_{i}$$
- Soft margin SVM
    - When data cannot be entirely separated, margin is maximized with allowance for some wrong classification.
    - The constraint is understood as the label of the datapoint being greater or equal to 1-c.
    - Why is the constraint condition required?
        - Merely maximizing the margin does not guarantee that the data is properly clasisfied.
        - The constraint forces datapoints to lie on the correct side of the hyperplane and necessitates balance between margin maximization and classification accuracy.
        - y can be either 1 or -1 : the inequality changes for each class: if y = -1, the constraint is
        
        $$y_{i}(\mathbf{w} \cdot \mathbf{x}_{i} + b) \geq \xi_{i} - 1$$

$$\textbf{Margin}=\frac{2}{||\mathbf{w}||}$$

- Margin
    - The shortest distance between datapoints (support vectors) on the boundary hyperplanes of each class.
    - The objective of SVM is to maximize the margin. This is equivalent to minimizing the weight vector $||\mathbf{w}||$.

$$\underset{\mathbf{w},b,\xi} \min \left ( \frac{1}{2}||\mathbf{w}||^2+C\sum_{i=1}^{n}\xi_{i} \right )$$

- Optimization with Lagrange Multiplier Method
    - LMM is a method to solve an optimization problem with constraints. For SVM, LMM attempts to minimize the weight vector while satisfying the constraint condition.
    - LMM converts the primary problem into a dual problem. This makes the problem easy to transform and simpler to calculate.
    - Choosing support vectors: Datapoints greater than zero are chosen as support vectors.
    - Kernel tricks: Easier to apply kernel functions.

### Calculations: 

$$\mathcal{L}(\mathbf{w},b,\xi,\alpha,\mu) = \frac{1}{2}||\mathbf{w}||^{2}+C\sum_{i=1}^{n} \xi_{i} - \sum_{i=1}^{n}\alpha_{i}(y_{i}(\mathbf{w} \cdot \mathbf{x}_{i}+b)-1+\xi_{i})-\sum_{i=1}^{n}\mu_{i}\xi_{i}$$

$$\underset{\alpha}\max\sum_{i=1}^{n}\alpha_{i}-\frac{1}{2}\sum_{i=1}^{n}\sum_{j=1}^{n}\alpha_{i}\alpha_{j}y_{i}y_{j}(\mathbf{x}_{i} \cdot \mathbf{x}_{j}) \\ 0 \leq \alpha_{i} \leq C, \sum_{i=1}^{n} \alpha_{i}y_{i}=0$$

$$\mathbf{w}=\sum_{i=1}^{n}\alpha_{i}y_{i}\mathbf{x}_{i}$$

$$b=y_{k}-\mathbf{w} \cdot \mathbf{x}_{k}$$

### SVM Process
- Dummy data:
    - Positive class: 1
    - x1, x2
    - x1 = (1,2), y = 1
    - x2 = (2,3), y = 1
    - Negative class: -1
    - x3 = (4,1), y = -1
    - x4 = (3,0), y = -1
- Create SVM hyperplane and calculate support vectors.
- Simple optimization problem: find weight vector and bias with constraint conditions, solve with LMM dual problem.
    - Support vector examples: a1 = 0.5, a2 = 0.3, a3 = 0.4, a4 = 0 (not a support vector)
- Calculate hyperplane
    - $x_1 = (1,2), \alpha_1 = 0.5, y = 1 \to (0.5, 1)$
    - $x_2 = (2,3), \alpha_2 = 0.3, y = 1 \to (0.6, 0.9)$
    - $x_3 = (4,1), \alpha_3 = 0.4, y = -1 \to (1.6, -0.4)$
    - $\mathbf{w} = (0.5, 1) + (0.6, 0.9) + (-1.6, -0.4) = (-0.5, 1.5)$
    - $b = y_1 - \mathbf{w} \cdot \mathbf{x}_1 = -1.5$
    - Final equation: $-\mathbf{w} * \mathbf{x} + b = 0: -0.5x_1 + 1.5x_2=1.5=0, x_2 = \frac{1}{3}x_1 + 1$
- The support vector values $\alpha_1 \sim \alpha_n$ are optimized with partial derivatives.

### SVM with Dummy Data: Code

In [ ]:
# Import necessary libraries

import numpy as np
from numpy import linalg
import matplotlib.pyplot as plt

In [ ]:
# Generate dataset
np.random.seed(42)
X1 = np.random.randn(20,2) + np.array([2,2])
X2 = np.random.randn(20,2) + np.array([-2,-2])

X = np.vstack([X1, X2])
y = np.hstack([np.ones(20), -np.ones(20)])

In [ ]:
# Visualize data
plt.scatter(X[:20,0], X[:20,1], color='b', label='1')
plt.scatter(X[20:,0], X[20:,1], color='r', label='-1')
plt.legend()
plt.show()

In [ ]:
# Define functions

# Linear kernel
def linear_kernel(x1, x2):
    '''
    Calculate the linear kernel between two vectors.
    '''
    return np.dot(x1, x2)

def fit_svm(X, y, C = 1.0, epoch = 1000):
    '''
    Fit the SVM model. Requires Lagrange multiplier alpha, normal weight vector of hyperplane w, and bias b.
    Factors:
    X: Training data (n_samples, n_features)
    y: Target classes(1 or -1)
    C: Penalty term
    
    Returns:
    w: Weight vector
    b: Bias
    alpha: Optimized Lagrange multiplier
    '''
    
    n_samples, n_features = X.shape
    alpha = np.zeros(n_samples) # Initialize alpha
    w = np.zeros(n_features) # Initialize weight vector
    b = 0 # Initialize bias
    
    K = np.zeros((n_samples, n_samples)) # Initialize kernel matrix
    for i in range(n_samples):
        for j in range(n_samples):
            K[i,j] = linear_kernel(X[i], X[j]) # Calculate kernel matrix (linear kernel)
    
    # Recursively optimize alpha
    for _ in range(epoch):
        for i in range(n_samples):
            for j in range(n_samples):
                alpha[i] += y[j] * alpha[j] * K[i,j]
        alpha = np.maximum(alpha, 0)
        
    # Calculate weight vector and bias
    
    for i in range(n_samples):
        w += alpha[i] * y[i] * X[i]
        b += y[i]
        b -= np.sum(alpha * y * K[i, :])
    b /= n_samples
    
    return w, b, alpha